## 1. 写在最前

>   nohup python3 xxx.py 10086 &

```
能够使用谷歌浏览器的开发者工具查看HTTP协议的通信过程 3个学生不清楚
能够知道可以使用多线程开发多任务版的tcp服务端程序    4个学生不清楚
能够知道web服务器发送响应报文的格式               1个学生不清楚
```

HTTP 协议作用： 浏览器 和 web服务器 之间传输 网页资源 的协议

底层基于 TCP 协议 实现的

URL 由 协议://、主机名、/资源路径、?查询参数 几部分构成

请求报文有 几部分构成: 请求行(方法、路径、版本\r\n)、头、空行、请求体

GET 和 POST的区别

响应报文有几部分构成:状态行(版本、状态码、说明\r\n)、头、响应体

常用状态码意义

```shell
ps aux | grep nginx # 得到 master 进程的 PID
sudo kill -9 PID
```

## 2. web 服务器

需求:   

​	之前的代码中端口都是在代码中写死的、如果用户需要更改端口情况，该代码

​	代码的可维护性、可扩展性角度来说(可变的做成可配置的  以免影响其他的代码)

实现思路:

​	1 把程序所需的代码通过命令指定  python3 web.py 8080

​	2 在 web.py 程序内部需要接收用户指定的端口号

​	3 在程序内部可以通过 sys.argv获取当前程序在启动时的命令行参数

​		是一个列表、每个元素都是字符串

​	

实现:

```python
import socket

#  1.0 当接收到用户请求时 返回响应报文< helloworld作为响应体>
#  2.0 当接收到用户请求时 返回响应报文<固定图片作为响应体>
#  3.0 根据用户的具体资源路径请求 返回相应报文<指定资源作为响应体>
#  4.0 多线程实现　　　　作业用协程实现  handler处理者　　处理逻辑
import threading
import time

import sys


class HTTPServer(object):
    def __init__(self, port=80):
        # 1 创建监听套接字 设置套接字重用选项  绑定监听
        listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        listen_socket.bind(('', port))
        listen_socket.listen(128)
        self.listen_socket = listen_socket

    def start(self):
        while True:
            # 2 接受用户的连接请求
            server_client_socket, client_address = self.listen_socket.accept()
            print("接受到了来自%s的连接请求" % str(client_address))
            # 把所有跟用户请求处理相关的逻辑放到client_request_handler函数中 子线程运行
            # 一旦有新用户连接到服务器 我们就为这个用户创建一个线程处理逻辑
            thd = threading.Thread(target=self.client_request_handler, args=(server_client_socket,))
            thd.start()

    # 处理客户端请求的函数
    @staticmethod
    def client_request_handler(server_client_socket):
        # 3 接收用户的HTTP请求报文数据  解析处理 获取用户的目的 一定要接收
        # 3.1 接收请求报文
        http_request_data = server_client_socket.recv(4096)
        # 3.2 如果数据为空 意味着浏览器中途关闭了
        if not http_request_data:
            print("客户端已经关闭了")
            server_client_socket.close()
            return
        # 3.3 从请求报文中解析出请求行的  用户的资源路径
        path_info = http_request_data.decode().split("\r\n")[0].split(" ")[1]
        print("接收到了用户的请求 %s" % path_info)

        # 3.4 如果用户请求路径是　/  －－－－－－－－> 指定到主页
        if path_info == '/':
            path_info = '/grand.html'

        # 4 准备数据  以HTTP响应报文格式发送给浏览器
        # 响应体中可以放任意类型的数据　　发给浏览器
        try:
            # 可能会抛出异常的代码
            # 4.1 准备响应报文
            # http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\nhello world'
            http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\n'
            # 4.2 从用户指定的文件读取出对应的　资源数据
            # with open("static/xjj.jpg", "rb") as file:
            with open("static" + path_info, "rb") as file:
                response_body = file.read()  # 读出来就是字节数据  rb
        except Exception as e:
            # 如果有异常执行这里的代码
            print("%s" % str(e))
            http_response_data = 'HTTP/1.1 404 Not Found\r\nServer: PWS1.0\r\n\r\n'
            with open("static/404.html", "rb") as file:
                response_body = file.read()  # 读出来就是字节数据  rb
        else:
            # 如果没有发生异常就会执行这里
            pass
        finally:
            # 不管有没有发生异常都会执行这里
            # 4.3 将报文拼接完成发送给浏览器
            server_client_socket.send(http_response_data.encode() + response_body)

            # 5 关闭套接字
            server_client_socket.close()


def main():
    # xxx.py 8080
    # print(sys.argv)  # 获取当前程序启动时的命令行参数 的列表 每个元素都是字符串
    # print("获取当前程序的端口好", sys.argv[1])
    # 如果当前的参数数量不是２个　　或者　第一个参数不是由数字构成
    if len(sys.argv) !=2 or not sys.argv[1].isdigit(): #　判断当前参数的和数量
        print("参数使用错误　python3 web.py 8080")
        return

    port = int(sys.argv[1])

    http_server = HTTPServer(port)
    http_server.start()
    # 实例对象http_server 类对象HTTPServer
if __name__ == '__main__':
    main()
    # 功能      类和对象(封装功能和属性)
    # 吃(狗,翔) 狗.吃(翔)
```

## 3. HTML 知识



### 3.1 简介 

HTML 作用: 编写网页的技术

​	超文本标记语言

​	超文本: 能够包含非普通文本的数据  比如音频视频图片

​	标记: 标签      

​			双标签<标签名 x属性="y值">内容</标签名>  

​			单标签<标签名 x属性="y值">

```html
<head>标签中都是存在的 网页基础声明
<body>标签中都是用以显示 网页页面的内容
```

### 3.2 常用标签

​	div img a p br hx

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    
    <!-- 常用标签介绍 -->
    <!-- 1 标题标签 -->
    <h1>这是一级标题</h1>
    <h2>这是二级标题</h2>
    <h3>这是三级标题</h3>
    <h6>这是六级标题</h6>

    <!-- 2 区块 布局标签 -->
    <div>这是一个区块</div>

    <!-- 3 段落标签 -->
    <p>曾经我喜欢一个人, 这个人叫小芳，现在我还是喜欢一个人，所以我单身了</p>
    这是一个普通文字
    <br>
    这是一个普通文字
    
    <!-- 4 图片标签 src指定图片所在的路径  alt 指定图片不能显示时的所显示的文本-->
    <img src="img/i1111.jpg" alt="这是一张很帅气的图片">

    <!-- 5 超链接 href 指定超链接所需要跳转到的目的网址  target="_blank"新标签页打开网页-->
    <a href="http://www.jd.com">点击我有惊喜</a>

    <!-- 6 案例: 点击图片跳转到网址 -->
    <a href="http://www.baidu.com/s?wd=haha" target="_blank">
        <img src="img/i2.jpg" alt="">
    </a>
</body>
</html>
```



### 3.3 资源路径

​	绝对路径 win: E:/data/static/xjj1.jpg  Linux: /home/python/Desktop/xjj.jpg

​	相对路径:  ./xxx  ../

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    
    <!-- 1 绝对路径 /  E:/data/dafdsf/dsafds 不带空格和中文-->
    <img src="/Users/PC/Desktop/HTML_CSS/img/i3.jpg" alt="">
    <!-- 2 相对路径 ./ ../ -->
    <img src="./img/i4.jpg" alt="">
    <!-- 3 互联网绝对地址 外链 -->
    <img src="https://rpic.douyucdn.cn/live-cover/appCovers/2019/06/08/5278899_20190608075810_big.jpg" alt="">
    <img src="https://tlias3.boxuegu.com/images/login/banner-2.png" alt="">
</body>
</html>
```

### 3.4 列表标签

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    
    <!-- 1 无序列表 unordered list ul-->
    <ul>
        <li>抽烟</li>
        <li>喝酒</li>
        <li>烫头</li>
    </ul>
    <!-- 2 有序列表 ordered list ol-->
    <!-- ol>li*3 -->
    <ol>
        <li>北京</li>
        <li>深圳</li>
        <li>上海</li>
    </ol>
</body>
</html>
```



### 3.5 表格标签

&nbsp;  可以在编写html 时当做空格 使用`&nbsp;`

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    <!-- 表格标签 table-->
    <!-- 姓名  年龄  身高 表头 table header-->
    <!-- 张三  28   170  表数据 table data-->
    <!-- 李四  26   180 -->
    哈哈 哈哈
    <p>哈哈&nbsp; &nbsp; 哈哈</p>
    <table>
        <!-- tr 表示一行数据 表格标签第一行是表头
            一个表头字段用户用一对th表示
            后续都是表数据 一个表数据字段用一对td表示
        -->

        <tr>
            <th>姓名</th>
            <th>年龄</th>
            <th>身高</th>
        </tr>
        <tr>
            <td>张三</td>
            <td>28</td>
            <td>170</td>
        </tr>
        <tr>
            <td>李四</td>
            <td>26</td>
            <td>180</td>
        </tr>
    </table>
</body>
</html>
```



### 3.6 表单标签

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    <!-- 注册页面 -->
    <!-- 表单标签 form       里面放 表单元素标签-->
    <form action="">
        <!-- 1 文本标记标签 -->
        <label for="name1">姓名:</label>
        <!-- 2 文本text输入框标签 -->
        <input id="name1" type="text"> <br>
        <!-- 3 密码框标签 -->
        <label for="">密码:</label>
        <input type="password"><br>

        <!-- 4 多选框-->
        <label for="">爱好:</label>
        <input type="checkbox">聊天
        <input type="checkbox">打球
        <input type="checkbox">唱歌<br>

        <!-- 5 单选框 把所有的单选框设置为同样的 name-->
        <label for="">性别:</label>
        <input name="sex" type="radio">男
        <input name="sex"type="radio">女
        <input name="sex" type="radio">保密<br>

        <!-- 6 文件上传 -->
        <label for="">请选择你的照片:</label>
        <input type="file"><br>
        <!-- 7 提交submit按钮  上传数据到服务器；   重置按钮 reset  清空所有数据-->
        <input type="submit">
        <input type="reset">
        <!-- <input type="button"> -->
        <button>点我呀</button><br>

        <!-- 8 多行文本框 -->
        <br>
        <label for="">读后感</label>
        <textarea name="" id="" cols="30" rows="10"></textarea><br>
        <!-- 9 下拉菜单 -->

        <label for="">请选择你所在的城市:</label>
        <select name="" id="">
            <option value="">北京</option>
            <option value="">上海</option>
            <option value="">深圳</option>
        </select>
    </form>
</body>
</html>
```



### 3.7 表单提交

​	form 表单标签

​	表单内部的标签 表单元素标签

​	设置form 表单标签的

​		 action 属性指定后台接口地址 URL

​		method 属性指定提交方式

​	表单元素表的 name 属性

​		value属性   如果是输入框就不需要设置该属性

​		其他则	select、单选、多选 则需要设置

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
</head>
<body>
    <!-- 注册页面 -->
    <!-- 表单标签 form       里面放 表单元素标签-->
    <!-- 表单提交  设置1 : action 属性指定后台服务器的接口地址  URL method 属性指定提交数据的方式-->
    <!-- <form action="http://172.16.51.151:8888/register.html" method="GET"> -->
    <form action="http://172.16.51.151:8888/register.html" method="POST">
        <!-- 设置 2: 给所有标签设置 name 属性和 value<输入框> -->
        <!-- 1 文本标记标签 -->
        <label for="">姓名:</label>
        <!-- 2 文本text输入框标签 -->
        <input type="text" name="name"> <br>
        <!-- 3 密码框标签 -->
        <label for="">密码:</label>
        <input type="password" name="passwd"><br>

        <!-- 4 多选框-->
        <label for="">爱好:</label>
        <input type="checkbox" name="love" value="chat">聊天
        <input type="checkbox" name="love" value="ball">打球
        <input type="checkbox" name="love" value="sing">唱歌<br>

        <!-- 5 单选框 把所有的单选框设置为同样的 name-->
        <label for="">性别:</label>
        <input name="sex" type="radio" value="man">男
        <input name="sex"type="radio" value="woman">女
        <input name="sex" type="radio" value="nicai">保密<br>

        <!-- 6 文件上传 -->
        <label for="">请选择你的照片:</label>
        <input type="file" name="file"><br>


        <!-- 8 多行文本框 -->
        <br>
        <label for="">读后感</label>
        <textarea name="feeling" id="" cols="30" rows="10"></textarea>
        <!-- 9 下拉菜单 -->
        <br>
        <label for="">请选择你所在的城市:</label>
        <select name="address" id="">
            <option value="beijing">北京</option>
            <option value="shanghai">上海</option>
            <option value="shenzhen">深圳</option>
        </select>

                <!-- 7 提交submit按钮  上传数据到服务器；   重置按钮 reset  清空所有数据-->
                <input type="submit">
                <input type="reset">
                <!-- <input type="button"> -->
                <button>点我呀</button>
    </form>
</body>
</html>
```



GET 方式提交的数据示例

```http
GET /register.html?name=pc&passwd=12312&love=chat&sex=on HTTP/1.1
Host: 172.16.51.151:8888
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3
Accept-Encoding: gzip, deflate
Accept-Language: zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7
```

POST 方式提交的实例

```http
POST /register.html HTTP/1.1
Host: 172.16.51.151:8888
Connection: keep-alive
Content-Length: 38
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
Origin: null
Content-Type: application/x-www-form-urlencoded
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3
Accept-Encoding: gzip, deflate
Accept-Language: zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7

name=123&passwd=12312&love=chat&sex=on
```

## 4. CSS 知识

>   HTML 内容标准  基本内容
>
>   CSS   样式标签   布局、美化
>
>   JS      行为标准    行为动作 表单验证 

### 4.1 作用

​	美化网页

选择器目的: 就是为了找到标签的手段

```css
选择器名称 {
    属性名1: 值1;
    属性名2: 值2;
    ...
}
```



### 4.2 CSS 三种引入方式

my.css文件

```css
div{
    width: 100px;
    height: 100px;
    background: green;
}
```

html文件

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>

    <!-- 2 内嵌式 在 head 标签中新建一对 style 标签 将选择器样式写在其中   学习用
    优点: 当前文件中重用较好  缺点: 别的文件不能使用这个样式
    -->
    <!-- <style>
        div{
            width: 100px;
            height: 100px;
        }
    </style> -->

    <!-- 3 外链式 将 css 文件和 html 文件进行了分离 如果在 html 中需要使用就引入 css 文件即可 
    优点: 重用性最高   工作用法
    -->
    <link rel="stylesheet" href="./ my.css">
</head>
<body>
    <!-- 1 行内式 通过标签的 style 属性指定 css 样式  
        优点: 简单 缺点:重用性差      优先级比较高 -->
    <div style="background: red;">这是一个段落</div>
    <div>这是一个段落</div>
</body>
</html>
```

### 4.3 CSS 选择器-重点

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
    <style>
        /* CSS选择器: 找标签 给标签 加样式 */
        /*1  标签选择器 根据标签的名字找到标签加样式 */
        div{
            width: 100px;
            height: 100px;
            background: pink;
        }
        /*2  类选择器 根据标签的class属性找到标签加样式 选择器名以.开始*/
        .div2{
            background: red;
        }
        /*3  伪类选择器 选择器:hover  悬停效果*/
        img:hover{
            width: 220px;
            height: 220px;
        }
        /*4 后代选择器 找嵌套前面标签里面的 标签   格式:选择器 后代选择器 ... */
        /* div img 找嵌套在 div中 img   */
        /* 找嵌套在 div 中 a 中 img 标签 */
        div a img{
            width: 220px;
            height: 220px;
        }
        /*5 组选择器 通知给一组标签加统一的样式 格式: 选择器1,选择器2,选择器3,...*/
        p,span,ul{
            background: yellow;
        }
        /*6 id选择器 在 CSS 中 id选择器不常用 在 JS/jquery中   格式: #选择器 */
        /* 根据标签的 id 查找   规范:id 在整个网页中是唯一 */
        #l1,#l2{
            background: greenyellow;
        }
        /* 大多数选择器  都是可以组合在一起使用 */
    </style>
</head>
<body>
    
    <div>这是一个区块</div>
    <div class="div2">这是另一个区块</div>
    <img src="./img/i5.jpg" alt="">
    <p>这是一个段落</p>
    <span>这是一个段落</span>
    <ul>
        <li id="l1">元素 1</li>
        <li id="l2">元素 2</li>
        <li>元素 3</li>
        <li>元素 4</li>
    </ul>
    <div>
        <a href="">
            <img src="./img/i5.jpg" alt="">
        </a> 
    </div>
</body>
</html>
```



### 4.4 CSS 文字属性

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
    <style>
        #p1{
            /* 字体颜色 */
            color: blue;
            /* 字体大小 */
            font-size: 30px;
            /* 字体类型 */
            font-family: 'Microsoft Yahei';
            /* 是否加粗 */
            /* font-weight: bold; */
            font-weight: normal;
            /* 行高 */
            line-height: 30px;
            /* 文字下划线 */
            text-decoration: underline;
            /* 文本对齐 */
            text-align: left;
            /* text-align: right; */
            /* text-align: center; */
            /* 首行缩进 */
            /* text-indent: 60px; */
            text-indent: 2em; /*2个字宽度  */
        }
        a{
            text-decoration: none;
        }
        table{
            /* 将表格的边线进行合并 */
            border-collapse: collapse;
        }
    </style>
</head>
<body>
    <a href="">点击我惊喜</a>
    <!-- alt z 超长一行换成多行显示 -->
    <p id="p1">曾经我喜欢一个，显示还是喜欢一个人。原来喜欢的人叫小红,后来我们就在一起了，但是旁边有一个王木匠，于是他们一起私奔了，于是我就一个人。曾经我喜欢一个，显示还是喜欢一个人。原来喜欢的人叫小红,后来我们就在一起了，但是旁边有一个王木匠，于是他们一起私奔了，于是我就一个人。</p>
    <p>曾经我喜欢一个，显示还是喜欢一个人。原来喜欢的人叫小红,后来我们就在一起了，但是旁边有一个王木匠，于是他们一起私奔了，于是我就一个人。</p>
    <table border="1px">
        <tr>
            <th>姓名</th>
            <th>年龄</th>
        </tr>
        <tr>
            <td>张三</td>
            <td>22</td>
        </tr>
    </table>
</body>
</html>
```



### 4.5 CSS 盒子模型

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Document</title>
    <style>
        #div0{
            /* 1 内边距  当前盒子边框和内部内容的距离 */
            padding: 10px;
        }
        #div1{
            width: 100px;
            height: 100px;
            background: red;
            /* 2 边框  当前盒子的厚度 实线 黄色*/
            border: 10px solid yellow;
            /* 3 外边距 当前盒子边框和外界的距离 */
            margin: 50px;
        }
        #div2{
            width: 100px;
            height: 100px;
            background: green;
            margin: 50px;
        }
        /* 如果外部盒子设置内边距  内部设置了外边距 内部边框和外部边框的距离就是两者之和 */
    </style>
</head>
<body>
    <div id="div0">
        <div id="div1"></div>
        <div id="div2"></div>
    </div>
</body>
</html>
```

>   下次上课 还会再说一说   padding border margin 的其他用法